# Exploring Fraud Detection With Neo4j & Graph Data Science

__You can run this notebook to reset things before re-running the primary demo.__

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neo4j-product-examples/demo-fraud-detection-with-p2p/blob/main/abridged-demo/stage.ipynb)
![Neo4j version](https://img.shields.io/badge/Neo4j->=4.4.9-brightgreen)
![GDS version](https://img.shields.io/badge/GDS->=2.1-brightgreen)
![GDS Python Client version](https://img.shields.io/badge/GDS_Python_Client->=1.4-brightgreen)


## Notebook Setup <a name="p0"></a>

In [1]:
%%capture
%pip install graphdatascience python-dotenv

In [2]:
import pandas as pd
from graphdatascience import GraphDataScience
from dotenv import load_dotenv
import os
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 50)

### Connect to Neo4j Graph Data Science
Create a `db-credentials.env` from template the template and fill in credentials as oppropriate.


documentation: [https://neo4j.com/docs/graph-data-science-client/current/getting-started/#_import_and_setup](https://neo4j.com/docs/graph-data-science-client/current/getting-started/#_import_and_setup)

In [3]:
load_dotenv('db-credentials.env', override=True)

# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'),
          os.getenv('NEO4J_PASSWORD')),
    aura_ds=eval(os.getenv('AURA_DS').title()))

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")
PROJ_NAME = 'proj'

### Helper Functions

In [4]:
def clear_graph_by_name(g_name):
    if gds.graph.exists(g_name).exists:
        g = gds.graph.get(g_name)
        gds.graph.drop(g)

def clear_all_graphs():
    g_names = gds.graph.list().graphName.tolist()
    for g_name in g_names:
        g = gds.graph.get(g_name)
        gds.graph.drop(g)

## Clean Up
This section will help clean all the additional graph elements and properties created in the above workflow.

In [5]:
# delete created relationships
gds.run_cypher('MATCH (:User)-[r:P2P_WITH_SHARED_CARD]->() DELETE r')
gds.run_cypher('MATCH (:User)-[r:AGGREGATED_P2P]->() DELETE r')

""


In [6]:
gds.run_cypher('MATCH (:User)-[r:AGGREGATED_P2P]->() DELETE r')

""


In [7]:
# remove created node Labels
gds.run_cypher('MATCH (u:FlaggedUser) REMOVE u:FlaggedUser')
gds.run_cypher('MATCH (u:FraudRiskUser) REMOVE u:FraudRiskUser')
gds.run_cypher('MATCH (u:FraudSharedId) REMOVE u:FraudSharedId')
gds.run_cypher('MATCH (u:PredictedFraudRisk) REMOVE u:PredictedFraudRisk')

""


In [8]:
# remove created node properties
gds.run_cypher('''
    MATCH (n)
    REMOVE n.wccId,
        n.sharedIdsDegree,
        n.predictedProbability,
        n.partOfCommunity,
        n.p2pSharedCardPageRank,
        n.p2pSharedCardDegree,
        n.p2pSentWeightedPageRank,
        n.p2pSentWeightedDegree,
        n.p2pSentPageRank,
        n.p2pSentDegree,
        n.p2pReversedSharedCardPageRank,
        n.p2pReversedSharedCardDegree,
        n.p2pReceivedWeightedPageRank,
        n.p2pReceivedWeightedDegree,
        n.p2pReceivedPageRank,
        n.p2pReceivedDegree,
        n.louvainCommunityId,
        n.ipDegree,
        n.fraudRiskRatio,
        n.fraudRiskDegree,
        n.fraudRisk,
        n.flaggedRatio,
        n.flaggedDegree,
        n.deviceDegree,
        n.degree,
        n.communitySize,
        n.cardDegree
''')

""
